In [15]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

In [16]:
# Define constants
num_classes = 8
train_data_dir = 'train'
validation_data_dir = 'validation'
img_height, img_width = 640, 640
batch_size = 6
num_train_samples = 350
num_validation_samples = 100
epochs = 6

In [17]:
# Load Inception V3 model
base_model = InceptionV3(weights='imagenet', include_top=False)

# Add custom output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)  # Add a final softmax layer for classification

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# Preprocess the input data
train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_v3.preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_v3.preprocess_input)


87910968/87910968 [==============================] - 1s 0us/step


In [18]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

# Train the model
history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

In [19]:
# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print("Validation accuracy: {:.2f}%".format(accuracy * 100))

Found 2193 images belonging to 8 classes.
Found 621 images belonging to 8 classes.


In [20]:
# Prediction
Y_pred = model.predict(validation_generator)
y_pred = np.argmax(Y_pred, axis=1)

# Get class labels
class_labels = list(validation_generator.class_indices.keys())

# Print classification report
print("Classification Report:")
print(classification_report(validation_generator.classes, y_pred, target_names=class_labels))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(validation_generator.classes, y_pred))

# Save the trained model
model.save('Inceptionv_model.h5')


Epoch 1/6
366/366 [==============================] - 214s 496ms/step - loss: 1.3516 - accuracy: 0.5062 - val_loss: 1.1305 - val_accuracy: 0.6329
Epoch 2/6
366/366 [==============================] - 173s 473ms/step - loss: 0.9390 - accuracy: 0.6480 - val_loss: 1.4149 - val_accuracy: 0.5137
Epoch 3/6
366/366 [==============================] - 174s 474ms/step - loss: 0.8030 - accuracy: 0.7073 - val_loss: 0.9162 - val_accuracy: 0.6586
Epoch 4/6
366/366 [==============================] - 171s 467ms/step - loss: 0.6846 - accuracy: 0.7460 - val_loss: 0.8969 - val_accuracy: 0.6667
Epoch 5/6
366/366 [==============================] - 171s 466ms/step - loss: 0.5701 - accuracy: 0.7943 - val_loss: 2.2523 - val_accuracy: 0.6570
Epoch 6/6
366/366 [==============================] - 174s 476ms/step - loss: 0.4889 - accuracy: 0.8185 - val_loss: 2.6431 - val_accuracy: 0.5507


In [2]:
from tensorflow.keras.models import load_model
import os

# Load the saved model
loaded_model = load_model('Inceptionv_model.h5')

import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# Load the saved model
loaded_model = tf.keras.models.load_model('Inceptionv_model.h5')

# Load and preprocess the input image
img_path = '/content/bacterial_leaf_blight (1).JPG'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = tf.keras.applications.inception_v3.preprocess_input(img_array)

# Make prediction
predictions = loaded_model.predict(img_array)
predicted_class = np.argmax(predictions)

# Get the class name corresponding to the predicted class index
predicted_class_name = class_labels[predicted_class]

# Display the input image
plt.imshow(img)
plt.axis('off')
plt.title('Input Image')
plt.show()

# Print predicted class index and its corresponding class name
print("Predicted Class Index:", predicted_class)
print("Predicted Class Name:", predicted_class_name)

OSError: No file or directory found at Inceptionv_model.h5